# 🤺 Fence demo

This notebook demonstrates the use of various classes in this package. The core intent of this package is to provide a framework for interacting with language models in a way that is modular, extensible, and easy to use. The core classes are:
- `PromptTemplate`: A template for a prompt that can be rendered with a dictionary of variables (or using keywords). It wraps around jinja2.Template, but adds some additional functionality.
- `LLM`: A wrapper around a language model. 
- `Link`: Atomic LLM interaction wrapper. Takes in a PromptTemplate and a language model. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `TransformationLink`: A wrapper around a `function` that takes a dictionary of variables and returns a dictionary of variables. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `Chain`: A collection of links that are invoked in the right order based on the input and output keys for each Link.
- `LinearChain`: A sequence of links that are invoked in the order they are passed in.

Through these classes, we can interact with LLMs in varying degrees of complexity:
1. Just call the model directly
2. Use a `PromptTemplate` for reusability and abstraction
3. Use a `Link` for atomic LLM interaction
4. Use a `Chain` to execute a collection of `Links`

Below, we'll go through each of these levels of complexity and provide examples.

**Note** This notebook assumes you have access to AWS Bedrock, as we use Bedrock's Claude-instant model to fuel our LLM interactions.

In [1]:
from pathlib import Path
current_dir = Path('.').resolve().parents[0]
import sys
sys.path.append(str(current_dir))

In [2]:
%load_ext autoreload
from fence.models.claude import ClaudeInstant
from fence.models.claude3 import Claude35Sonnet
from fence.models.gpt import GPT4o
from fence.templates.string import StringTemplate
from fence.templates.messages import MessagesTemplate, Messages, Message
from fence.links import Link, TransformationLink
from fence.chains import Chain, LinearChain

## ⚙️ Setting up

In [3]:
# Get our model
model = Claude35Sonnet(source='demo_notebook', region='us-east-1')

### 🪨 Level 1 - Just call the damn thing

In [4]:
# Use the invoke method to call the model
model.invoke('Why is the sky blue?')

"The sky appears blue due to a phenomenon called Rayleigh scattering. Here's a simplified explanation:\n\n1. Sunlight contains all colors of the visible spectrum.\n\n2. As sunlight enters Earth's atmosphere, it collides with gas molecules and other particles.\n\n3. These collisions cause the light to scatter in all directions.\n\n4. Blue light has a shorter wavelength and higher frequency than other colors in the visible spectrum, so it scatters more easily than other colors.\n\n5. This scattered blue light reaches our eyes from all directions, making the sky appear blue.\n\n6. Other colors, like red and yellow, have longer wavelengths and pass through the atmosphere more directly, which is why we see them more prominently during sunrise and sunset when sunlight travels through more of the atmosphere to reach our eyes.\n\nThe blue color is more pronounced during the day when the sun is high in the sky. At sunrise and sunset, when sunlight travels through more of the atmosphere, we see 

In [5]:
# Just call the damn thing
model('Why is the sky blue?')

"The sky appears blue due to a phenomenon called Rayleigh scattering. Here's a brief explanation:\n\n1. Sunlight contains all colors of the visible spectrum.\n\n2. As light travels through the Earth's atmosphere, it collides with gas molecules and other particles.\n\n3. These collisions cause the light to scatter in different directions.\n\n4. Blue light has a shorter wavelength and higher frequency than other colors in the visible spectrum, so it scatters more easily.\n\n5. This scattered blue light reaches our eyes from all directions, making the sky appear blue.\n\n6. Other colors with longer wavelengths (like red and yellow) don't scatter as much and pass through the atmosphere more directly.\n\n7. At sunrise and sunset, when sunlight travels through more of the atmosphere to reach our eyes, the blue light scatters away, allowing us to see more of the red and orange colors.\n\nThis is why the sky appears blue during most of the day but can take on different hues at dawn and dusk."

### 🔨 Level 2 - Use a PromptTemplate

In [6]:
# Initialize a prompt template
prompt_template = StringTemplate('Why is the sky {color}?')
print(prompt_template)

StringTemplate: Why is the sky {color}?


In [7]:
# Render it with a dictionary
print(prompt_template.render({'color': 'blue'}))

# Render it with keyword arguments
print(prompt_template.render(color='red'))

# Input dict takes precedence over keyword arguments
print(prompt_template.render(input_dict={'color': 'blue'}, color='red'))

Why is the sky blue?
Why is the sky red?
Why is the sky red?


In [8]:
# You can concatenate prompt templates, input variables are merged
prompt_template_sky = StringTemplate('Why is the sky {color}?')
prompt_template_grass = StringTemplate('Why is the grass {color}?')
prompt_template_dress = StringTemplate('I like a dress with {pattern}.')
combined_prompt_template = prompt_template_sky + prompt_template_grass + prompt_template_dress
print(combined_prompt_template)
print(combined_prompt_template.render({'color': 'blue', 'pattern': 'polka dots'}))

StringTemplate: Why is the sky {color}? Why is the grass {color}? I like a dress with {pattern}.
Why is the sky blue? Why is the grass blue? I like a dress with polka dots.


In [9]:
# You can customize the separator
base_template = StringTemplate('Why is the sky {color}?', separator=' FUNKY TOWN ')
additional_template = StringTemplate('Why is the grass {color}?')
print((base_template + additional_template).render({'color': 'blue'}))

Why is the sky blue? FUNKY TOWN Why is the grass blue?


In [10]:
# You can also use the MessagesTemplates for more complex prompts
messages = Messages(
    system='Respond in a {tone} tone',
    messages= [
        Message(role="user", content="Why is the sky {color}?"),
        # Equivalent to Message(role='user', content=Content(type='text', text='Why is the sky blue?'))
        # But Content can also be an image, etc.
    ]
)
messages_template = MessagesTemplate(
    source=messages
)
print(messages_template)
print(f"\nRendered: {messages_template.render(tone='sarcastic', color='blue')}")

MessagesTemplate: messages=[Message(role='user', content='Why is the sky {color}?')] system='Respond in a {tone} tone'

Rendered: messages=[Message(role='user', content='Why is the sky blue?')] system='Respond in a sarcastic tone'


### 🧠 Level 3 - Use Links (lol)

What are Links? In this context, they represent atomic components of LLM interaction. That means they should be able to be strung together to form a Chain, although they can be used independently as well.

In [11]:
# The simplest link is the Link class, which just takes a prompt template and a model
link = Link(template=prompt_template, llm=model)
print(link)

Link: <['color']> -> <state>


In [12]:
# Invoke it
link(color='blue') # Or, equivalently, link.run(input_dict={'color': 'blue'})

[2024-07-11 12:24:24] [INFO] ℹ️ [links.run:198] Executing unnamed Link


{'state': "The sky appears blue due to a phenomenon called Rayleigh scattering. Here's a simplified explanation:\n\n1. Sunlight contains all colors of the visible spectrum.\n\n2. As sunlight enters Earth's atmosphere, it collides with gas molecules and other particles.\n\n3. These collisions cause the light to scatter in all directions.\n\n4. Blue light has a shorter wavelength and higher frequency than other colors in the visible spectrum, so it scatters more easily.\n\n5. This scattered blue light reaches our eyes from all directions, making the sky appear blue.\n\n6. Other colors, with longer wavelengths, pass through the atmosphere more directly and are less scattered.\n\n7. During sunrise and sunset, sunlight travels through more of the atmosphere, scattering the blue light away. This allows more red and orange light to reach our eyes, creating the colorful skies we see at these times.\n\nIt's worth noting that the sky's appearance can change based on factors like pollution, humid

In [13]:
# By default, output is stored under 'state'. You can get a copy (e.g., for inspection of intermediate results) by passing a different output key
link = Link(template=prompt_template, llm=model, output_key='intermediate')
link.run(input_dict={'color': 'blue'})

[2024-07-11 12:24:32] [INFO] ℹ️ [links.run:198] Executing unnamed Link


{'state': "The sky appears blue due to a phenomenon called Rayleigh scattering. Here's a simplified explanation:\n\n1. Sunlight contains all colors of the visible spectrum.\n\n2. As light travels through the Earth's atmosphere, it collides with gas molecules and other particles.\n\n3. These collisions cause the light to scatter in all directions.\n\n4. Blue light has a shorter wavelength and higher frequency than other colors in the visible spectrum, so it scatters more easily than other colors.\n\n5. This scattered blue light reaches our eyes from all directions, making the sky appear blue.\n\n6. Other colors with longer wavelengths (like red and yellow) pass through the atmosphere more directly, which is why the sun appears yellow or red, especially during sunrise and sunset when light travels through more of the atmosphere.\n\nThe blue color is more pronounced during the day when the sun is high in the sky. At sunrise and sunset, the light travels through more of the atmosphere, sca

In [14]:
# You can name your links for easier debugging in logs
link = Link(template=prompt_template, llm=model, name='sky')
link.run(input_dict={'color': 'blue'})

[2024-07-11 12:24:39] [INFO] ℹ️ [links.run:198] Executing <sky> Link


{'state': "The sky appears blue due to a phenomenon called Rayleigh scattering. Here's a brief explanation:\n\n1. Sunlight contains all colors of the visible spectrum.\n\n2. As light travels through the Earth's atmosphere, it collides with gas molecules and other particles.\n\n3. These collisions cause the light to scatter in all directions.\n\n4. Blue light has a shorter wavelength and higher frequency than other colors in the visible spectrum, so it scatters more easily.\n\n5. This scattered blue light reaches our eyes from all directions, making the sky appear blue.\n\n6. Other colors with longer wavelengths (like red and yellow) don't scatter as much and pass through the atmosphere more directly.\n\n7. At sunrise and sunset, when sunlight travels through more of the atmosphere to reach our eyes, the blue light is scattered away, allowing us to see more of the red and orange wavelengths.\n\nThis is why the sky appears blue during most of the day but can take on different colors at d

In [15]:
# You can also build TransformationLinks, which take a function that transforms any input_dict into a specific output
def concatenate(input: dict):
    return f"{input['X']} and {input['Y']}"

concat_link = TransformationLink(
    input_keys=["X", "Y"], function=concatenate, output_key="C"
)

concat_link.run(input_dict={"X": "Hello", "Y": "World"})

[2024-07-11 12:24:46] [INFO] ℹ️ [links.run:120] Executing Transformation Link


{'state': 'Hello and World', 'C': 'Hello and World'}

### 🚀 Level 4 - Use Chains (lol again)

In [16]:
# You can also build Chains, which are just a sequence of links. There are two types of chains: LinearChain and Chain. 
# LinearChain is a sequence of links, while Chain is a collection of links that are invoked in the right order based on the input and output keys for each Link.

# This is a LinearChain #
#########################

# Build some links
link_opposite = Link(
    template=StringTemplate(
        "What's the opposite of {A}? Reply with a few words max."
    ),
    name = 'opposite',
    output_key="X",
)
link_synonym = Link(
    template=StringTemplate(
        "What's a synonym for {B}. Reply with one word."
    ),
    name='synonym',
    output_key="Y",
)
link_poem = Link(
    template=StringTemplate(
        "Write a poem about {state}. Return only the poem, beginning with the title."
    ),
    name='poem',
    output_key="Z",
)

# Now build a LinearChain
linear_chain = LinearChain(llm=model, links=[link_opposite, link_synonym, concat_link, link_poem])

# Run it
result = linear_chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})

# Get the output
print(result['state'])

[2024-07-11 12:24:46] [INFO] ℹ️ [links.run:198] Executing <opposite> Link
[2024-07-11 12:24:46] [INFO] ℹ️ [chains.run:265] 🔑 State keys: ['A', 'B', 'llm', 'state', 'X'] (added: ['X', 'state', 'llm'], removed: [])
[2024-07-11 12:24:46] [INFO] ℹ️ [links.run:198] Executing <synonym> Link
[2024-07-11 12:24:47] [INFO] ℹ️ [chains.run:265] 🔑 State keys: ['A', 'B', 'llm', 'state', 'X', 'Y'] (added: ['Y'], removed: [])
[2024-07-11 12:24:47] [INFO] ℹ️ [links.run:120] Executing Transformation Link
[2024-07-11 12:24:47] [INFO] ℹ️ [chains.run:265] 🔑 State keys: ['A', 'B', 'llm', 'state', 'X', 'Y', 'C'] (added: ['C'], removed: [])
[2024-07-11 12:24:47] [INFO] ℹ️ [links.run:198] Executing <poem> Link
[2024-07-11 12:24:53] [INFO] ℹ️ [chains.run:265] 🔑 State keys: ['A', 'B', 'llm', 'state', 'X', 'Y', 'C', 'Z'] (added: ['Z'], removed: [])
The Hopeful Outlaw

Behind iron bars, a heart still beats,
With dreams of change and new-found feats.
Though shadows of the past loom near,
Tomorrow's light shines bri

In [17]:
# A LinearChain will take the presence of the 'state' key into account when invoking the next link.
# A Chain will not. However, it has an extra 'feature' in the form of topological sorting. As long as a graph of links can be
# extracted from the chain, and the input keys (that are not generated in the chain) are given, the chain will invoke the links in the right order.

# This is a Chain #
###################
link_a = Link(
    template=StringTemplate(
        "Capitalize this word: {A}. Only respond with the capitalized version",
    ),
    name = 'opposite',
    output_key="X",
)
link_b = Link(
    template=StringTemplate(
        "What's a synonym for {B}. Reply with one word.", 
    ),
    name='superlative',
    output_key="Y",
)
link_c = Link(
    template=StringTemplate(
        "Combine {X} and {Y} and {C} in a meaningful sentence.", 
    ),
    name='sentence',
    output_key="Z",
)
chain = Chain(llm=model, links=[link_c, link_a, link_b]) # Note that we can pass the links in any order

# This is the sorted graph of links
chain._topological_sort()

[Link: superlative <['B']> -> <Y>,
 Link: opposite <['A']> -> <X>,
 Link: sentence <['X', 'C', 'Y']> -> <Z>]

In [18]:
# Now we can run it
try:
    result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})
except Exception as e:
    print(e)

The following input keys are required: {'A', 'C', 'B'}. Missing: {'C'}


In [19]:
# Woops, forgot something! There's no link that generates the 'C' key. We can pass it in though.
result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful", "C": "a dog"})
print(result['state'])

[2024-07-11 12:24:53] [INFO] ℹ️ [links.run:198] Executing <superlative> Link
[2024-07-11 12:24:53] [INFO] ℹ️ [links.run:198] Executing <opposite> Link
[2024-07-11 12:24:54] [INFO] ℹ️ [links.run:198] Executing <sentence> Link
The optimistic police officer adopted a retired K-9 dog, believing their shared experiences would make them the perfect team for community outreach programs.


In [20]:
# Cycles are not allowed
link_up = Link(
    template=StringTemplate(
        "Capitalize this word: {up}. Only respond with the capitalized version",
    ),
    name = 'up',
    output_key="down",
)
link_b = Link(
    template=StringTemplate(
        "What's a synonym for {down}. Reply with one word.", 
    ),
    name='down',
    output_key="up",
)
chain = Chain(llm=model, links=[link_up, link_b])
try:
    chain._topological_sort()
except Exception as e:
    print(e)

Cycle detected in the dependency graph.
